<a href="https://colab.research.google.com/github/crychenet/analysis_of_financion_report_WB/blob/master/OZON.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [59]:
import pandas as pd
import numpy as np
from copy import deepcopy

df = pd.read_excel("/content/Финансовый отчет по OZON 1_15 октября.xlsx")
df_cost_of_goods = pd.read_excel("/content/2023.10.09 Отчет по скидкам_111023.xlsx",
                                 skiprows=2)
article_report = deepcopy(df)
article_report.drop(columns=['SKU', 'Итого', 'Количество', 'Сборка заказа', 'Название товара или услуги', 'Склад отгрузки',
                   'Дата принятия заказа в обработку или оказания услуги',
                   'Номер отправления или идентификатор услуги', 'Дата начисления',
                   'Обработка отправления (Drop-off/Pick-up) (разбивается по товарам пропорционально количеству в отправлении)',
                   'Магистраль', 'Обратная магистраль', 'Индекс локализации', 'Ставка комиссии'], inplace=True)
article_report = article_report.groupby('Артикул').sum()


<ipython-input-59-ba15dd5ca146>:14: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  article_report = article_report.groupby('Артикул').sum()


In [41]:
article_report.columns

Index(['За продажу или возврат до вычета комиссий и услуг', 'Ставка комиссии',
       'Комиссия за продажу',
       'Последняя миля (разбивается по товарам пропорционально доле цены товара в сумме отправления)',
       'Обработка возврата',
       'Обработка отмененного или невостребованного товара (разбивается по товарам в отправлении в одинаковой пропорции)',
       'Обработка невыкупленного товара', 'Логистика', 'Обратная логистика'],
      dtype='object')

In [62]:
article_report['Итоговая логистика'] = article_report['Последняя миля (разбивается по товарам пропорционально доле цены товара в сумме отправления)'] + article_report['Обработка возврата'] + article_report['Обработка отмененного или невостребованного товара (разбивается по товарам в отправлении в одинаковой пропорции)'] + article_report['Обработка невыкупленного товара'] + article_report['Логистика'] + article_report['Обратная логистика']


In [69]:
article_report.reset_index(drop=False, inplace=True)

In [72]:
def get_number_of_sales(column):     # У number_of_sales и number_of_returns свои критерии!
    number_of_sales = df[(df['Артикул'] == column) & (
            df['За продажу или возврат до вычета комиссий и услуг'] > 0)]['Количество'].sum()
    return number_of_sales

def get_number_of_returns(column): # У number_of_sales и number_of_returns свои критерии!
    number_of_returns = df[
        (df['Артикул'] == column) & (df['За продажу или возврат до вычета комиссий и услуг'] < 0)]['Количество'].sum()
    return number_of_returns

article_report['Кол-во продаж'] = article_report['Артикул'].apply(get_number_of_sales)
article_report['Кол-во возвратов'] = article_report['Артикул'].apply(get_number_of_returns)

In [94]:
article_report.to_excel('article_report.xlsx')

In [87]:
article_report['Артикул-3'] = article_report['Артикул'].apply(lambda x: x[:-3])
article_report['Артикул-6'] = article_report['Артикул'].apply(lambda x: x[:-6])


In [90]:
article_report = pd.merge(article_report,df_cost_of_goods[['Артикул 1С', 'Настоящий СЕБЕК', 'КОЛЛЕКЦИЯ!!!', 'текущая РРЦ']], left_on='Артикул-3', right_on='Артикул 1С', how='left')